In [ ]:
import gym
import random
import numpy as np
from environment import GridWorld
import cv2

#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

from keras import layers
from skimage.color import rgb2gray
from skimage.transform import resize
from keras.models import Model
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
from collections import deque
from keras.optimizers import RMSprop
from keras import backend as K
from datetime import datetime
import os.path
import time
from keras.models import load_model
from keras.models import clone_model
from keras.callbacks import TensorBoard
from keras.models import model_from_json


from matplotlib import pyplot as plt
import pylab
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10


debug_mode = False
show_graph_every = 100
means = True
n_frames = 2
manual = False



FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('train_dir', 'tf_train_breakout',
                           """Directory where to write event logs and checkpoint. """)
tf.app.flags.DEFINE_string('restore_file_path',
                           'tf_train_breakout/breakout_model_breakout_scratch',
                           #'tf_train_breakout/breakout_model_testGridWordl1',
                           """Path of the restore file """)
tf.app.flags.DEFINE_integer('num_episode', 100000,
                            """number of epochs of the optimization loop.""")
tf.app.flags.DEFINE_integer('observe_step_num', 1,
                            """Timesteps to observe before training.""")
tf.app.flags.DEFINE_integer('epsilon_step_num', 1000000,
                            """frames over which to anneal epsilon.""")
tf.app.flags.DEFINE_integer('refresh_target_model_num', 10000,  # update the target Q model every refresh_target_model_num
                            """frames over which to anneal epsilon.""")
tf.app.flags.DEFINE_integer('replay_memory', 100000,  # takes up to 20 GB to store this amount of history data
                            """number of previous transitions to remember.""")
tf.app.flags.DEFINE_integer('no_op_steps', 1,
                            """Number of the steps that runs before script begin.""")
tf.app.flags.DEFINE_float('regularizer_scale', 0.01,
                          """L1 regularizer scale.""")
tf.app.flags.DEFINE_integer('batch_size', 64,
                            """Size of minibatch to train.""")
tf.app.flags.DEFINE_float('learning_rate', 0.0025,
                          """Number of batches to run.""")
tf.app.flags.DEFINE_float('init_epsilon', 1.0,
                          """starting value of epsilon.""")
tf.app.flags.DEFINE_float('final_epsilon', 0.1,
                          """final value of epsilon.""")
tf.app.flags.DEFINE_float('gamma', 0.01,
                          """decay rate of past observations.""")
tf.app.flags.DEFINE_boolean('resume', False,
                            """Whether to resume from previous checkpoint.""")
tf.app.flags.DEFINE_boolean('render', False,
                            """Whether to display the game.""")



In [ ]:
if manual:
    cv2.namedWindow('GUI')

ATARI_SHAPE = (5, 8, n_frames)  # input image size to model
ACTION_SIZE = 5


now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
now = 'testGridWordl2'

scores = []
averages = []
episodes = []

# 210*160*3(color) --> 84*84(mono)
# float --> integer (to reduce the size of replay memory)
def pre_processing(observe):
    return observe


def huber_loss(y, q_value):
    error = K.abs(y - q_value)
    quadratic_part = K.clip(error, 0.0, 1.0)
    linear_part = error - quadratic_part
    loss = K.mean(0.5 * K.square(quadratic_part) + linear_part)
    return loss


def atari_model():
    # With the functional API we need to define the inputs.
    frames_input = layers.Input(ATARI_SHAPE, name='frames')
    actions_input = layers.Input((ACTION_SIZE,), name='action_mask')
    '''
    # Assuming that the input frames are still encoded from 0 to 255. Transforming to [0, 1].
    #normalized = layers.Lambda(lambda x: x / 2, name='normalization')(frames_input)
    
    # "The first hidden layer convolves 16 8×8 filters with stride 4 with the input image and applies a rectifier nonlinearity."
    conv_1 = layers.convolutional.Conv2D(
        32, (2, 2), strides=(1, 1), activation='relu'
    )(frames_input)

    # "The second hidden layer convolves 32 4×4 filters with stride 2, again followed by a rectifier nonlinearity."
    conv_2 = layers.convolutional.Conv2D(
        32, (2, 2), strides=(2, 2), activation='relu'
    )(conv_1)
    '''
    
    # Flattening the second convolutional layer.
    conv_flattened = layers.core.Flatten()(frames_input)
    # "The final hidden layer is fully-connected and consists of 256 rectifier units."
    hidden = layers.Dense(256, activation='relu')(conv_flattened)
    # "The output layer is a fully-connected linear layer with a single output for each valid action."
    output = layers.Dense(ACTION_SIZE)(hidden)
    # Finally, we multiply the output by the mask!
    filtered_output = layers.Multiply(name='QValue')([output, actions_input])

    model = Model(inputs=[frames_input, actions_input], outputs=filtered_output)
    model.summary()
    optimizer = RMSprop(lr=FLAGS.learning_rate, rho=0.95, epsilon=0.01)
    # model.compile(optimizer, loss='mse')
    # to changed model weights more slowly, uses MSE for low values and MAE(Mean Absolute Error) for large values
    model.compile(optimizer, loss=huber_loss)
    return model



# get action from model using epsilon-greedy policy
def get_action(history, epsilon, step, model):
    '''
    print(history.shape)
    print('-------------------------------------------------------------')
    objecto = np.reshape(history[0,:,:,0], (5, 8))
    print(objecto)
    objecto = np.reshape(history[0,:,:,1], (5, 8))
    print(objecto)
    '''
    
    if np.random.rand() <= epsilon or step <= FLAGS.observe_step_num:
        if manual:
            key = cv2.waitKey(0)
            if key == ord('w'):
                return 4
            elif key == ord('s'):
                return 3
            elif key == ord('a'):
                return 2
            elif key == ord('d'):
                return 1
            else:
                return 0
        else:
            return random.randrange(ACTION_SIZE)
    else:
        q_value = model.predict([history, np.ones(ACTION_SIZE).reshape(1, ACTION_SIZE)])
        return np.argmax(q_value[0])


# save sample <s,a,r,s'> to the replay memory
def store_memory(memory, history, action, reward, next_history):
    memory.append((history, action, reward, next_history))


def get_one_hot(targets, nb_classes):
    return np.eye(nb_classes)[np.array(targets).reshape(-1)]


# train model by random batch
def train_memory_batch(memory, model, log_dir):
    mini_batch = random.sample(memory, FLAGS.batch_size)
    history = np.zeros((FLAGS.batch_size, ATARI_SHAPE[0],
                        ATARI_SHAPE[1], ATARI_SHAPE[2]))
    next_history = np.zeros((FLAGS.batch_size, ATARI_SHAPE[0],
                             ATARI_SHAPE[1], ATARI_SHAPE[2]))
    target = np.zeros((FLAGS.batch_size,))
    action, reward = [], []

    for idx, val in enumerate(mini_batch):
        history[idx] = val[0]
        next_history[idx] = val[3]
        action.append(val[1])
        reward.append(val[2])

    actions_mask = np.ones((FLAGS.batch_size, ACTION_SIZE))
    next_Q_values = model.predict([next_history, actions_mask])
    print(next_Q_values)

    # like Q Learning, get maximum Q value at s'
    # But from target model
    for i in range(FLAGS.batch_size):
        target[i] = reward[i] + FLAGS.gamma * np.amax(next_Q_values[i])

    action_one_hot = get_one_hot(action, ACTION_SIZE)
    target_one_hot = action_one_hot * target[:, None]

    #tb_callback = TensorBoard(log_dir=log_dir, histogram_freq=0,
    #                          write_graph=True, write_images=False)

    ''''''
    h = model.fit(
        [history, action_one_hot], target_one_hot, epochs=1,
        batch_size=FLAGS.batch_size, verbose=0)
        #batch_size=FLAGS.batch_size, verbose=0, callbacks=[tb_callback])

    #if h.history['loss'][0] > 10.0:
    #    print('too large')

    return h.history['loss'][0]

def train():
    
    
    env = GridWorld(show_graph_every, debug_mode, means)
    
    
    # deque: Once a bounded length deque is full, when new items are added,
    # a corresponding number of items are discarded from the opposite end
    memory = deque(maxlen=FLAGS.replay_memory)
    episode_number = 0
    epsilon = FLAGS.init_epsilon
    epsilon_decay = (FLAGS.init_epsilon - FLAGS.final_epsilon) / FLAGS.epsilon_step_num
    global_step = 0

    
    if FLAGS.resume:
        
        # load json and create model
        json_file = open(FLAGS.restore_file_path+'.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        # load weights into new model
        model.load_weights(FLAGS.restore_file_path+'.h5')
        
        optimizer = RMSprop(lr=FLAGS.learning_rate, rho=0.95, epsilon=0.01)
        # model.compile(optimizer, loss='mse')
        # to changed model weights more slowly, uses MSE for low values and MAE(Mean Absolute Error) for large values
        model.compile(optimizer, loss=huber_loss)
        
        print("Loaded model from disk")
        '''
        model = load_model('tf_train_breakout/breakout_model_20180610205843_36h_12193ep_sec_version_back.h5')#, custom_objects={'huber_loss': huber_loss})
        print("Loaded model from disk")
        #model = load_model(FLAGS.restore_file_path)
        '''
        #model = load_model(FLAGS.restore_file_path)
        # Assume when we restore the model, the epsilon has already decreased to the final value
        epsilon = FLAGS.final_epsilon
    else:
        model = atari_model()

    log_dir = "logs/{}/run-{}-log".format(FLAGS.train_dir, now)
    file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

    model_target = clone_model(model)
    model_target.set_weights(model.get_weights())

    while episode_number < FLAGS.num_episode:

        done = False
        # 1 episode = 5 lives
        step, score = 0, 0
        loss = 0.0
        observe = env.reset() 
        '''
        # this is one of DeepMind's idea.
        # just do nothing at the start of episode to avoid sub-optimal
        for _ in range(random.randint(1, FLAGS.no_op_steps)):
            observe, _, _, _, _ = env.step(1)
        '''
        # At start of episode, there is no preceding frame
        # So just copy initial states to make history
        state = pre_processing(observe)
        history = np.stack((state, state), axis=2)
        history = np.reshape([history], (1, 5, 8, n_frames))

        while not done:

            # get action for the current history and go one step in environment
            action = get_action(history, epsilon, global_step, model_target)
            # change action to real_action
            real_action = action #+ 1

            # scale down epsilon, the epsilon only begin to decrease after observe steps
            if epsilon > FLAGS.final_epsilon and global_step > FLAGS.observe_step_num:
                epsilon -= epsilon_decay

            #observe, reward, done, info = env.step(real_action)
            observe_past, reward, total_reward, observe, done = env.step(real_action)
            # pre-process the observation --> history
            next_state = pre_processing(observe)
            next_state = np.reshape([next_state], (1, 5, 8, 1))
            next_history = np.append(next_state, history[:, :, :, :n_frames-1], axis=3)
            
            # save the statue to memory, each replay takes 2 * (84*84*4) bytes = 56448 B = 55.125 KB
            store_memory(memory, history, action, reward, next_history)  #

            # check if the memory is ready for training
            if global_step > FLAGS.observe_step_num:
                loss = loss + train_memory_batch(memory, model, log_dir)
                # if loss > 100.0:
                #    print(loss)
                if global_step % FLAGS.refresh_target_model_num == 0:  # update the target model
                    model_target.set_weights(model.get_weights())

            score += reward
            history = next_history

            #print("step: ", global_step)
            global_step += 1
            step += 1

            if done:
                if global_step <= FLAGS.observe_step_num:
                    state = "observe"
                elif FLAGS.observe_step_num < global_step <= FLAGS.observe_step_num + FLAGS.epsilon_step_num:
                    state = "explore"
                else:
                    state = "train"
                print('state: {}, episode: {}, score: {}, global_step: {}, avg loss: {}, step: {}, memory length: {}'
                      .format(state, episode_number, score, global_step, loss / float(step), step, len(memory)))

                if episode_number % 100 == 0 or (episode_number + 1) == FLAGS.num_episode:
        
                #if episode_number % 1 == 0 or (episode_number + 1) == FLAGS.num_episode:  # debug
                    file_name = "breakout_model_{}".format(now)
                    model_path = os.path.join(FLAGS.train_dir, file_name)
                    
                    # serialize model to JSON
                    model_json = model.to_json()
                    with open(model_path+'.json', "w") as json_file:
                        json_file.write(model_json)
                    # serialize weights to HDF5
                    model.save_weights(model_path+'.h5')
                    #model.save(model_path)

                # Add user custom data to TensorBoard
                loss_summary = tf.Summary(
                    value=[tf.Summary.Value(tag="loss", simple_value=loss / float(step))])
                file_writer.add_summary(loss_summary, global_step=episode_number)

                score_summary = tf.Summary(
                    value=[tf.Summary.Value(tag="score", simple_value=score)])
                file_writer.add_summary(score_summary, global_step=episode_number)

                episode_number += 1
                PlotModel(score, episode_number)

    file_writer.close()




def PlotModel(score, episode_number):
    scores.append(score)
    averages.append(sum(scores[-50:]) / len(scores[-50:]))
    episodes.append(episode_number)
    pylab.plot(episodes, scores, 'b')
    pylab.plot(episodes, averages, 'r')
    pylab.ylabel('Score', fontsize=18)
    pylab.xlabel('Games', fontsize=18)
    file_name = "breakout_model_{}".format(now)
    name = file_name + '.png'
    try:
        if not os.path.exists('training_images'): os.makedirs('training_images')
        pylab.savefig('training_images/'+name)

    except OSError as e:
        print(e)
        pass
    return             
                
     
                
                
                
def main(argv=None):
    train()
    #test()


if __name__ == '__main__':
    tf.app.run()



In [ ]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


from scores.score_logger import ScoreLogger

ENV_NAME = "CartPole-v1"

GAMMA = 0.95
LEARNING_RATE = 0.001

MEMORY_SIZE = 1000000
BATCH_SIZE = 20

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.995


class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX

        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)

        self.model = Sequential()
        self.model.add(Dense(24, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(24, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def experience_replay(self):
        if len(self.memory) < BATCH_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0)
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)


def cartpole():
    env = gym.make(ENV_NAME)
    score_logger = ScoreLogger(ENV_NAME)
    observation_space = env.observation_space.shape[0]
    action_space = env.action_space.n
    dqn_solver = DQNSolver(observation_space, action_space)
    run = 0
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        step = 0
        while True:
            step += 1
            #env.render()
            action = dqn_solver.act(state)
            state_next, reward, terminal, info = env.step(action)
            reward = reward if not terminal else -reward
            state_next = np.reshape(state_next, [1, observation_space])
            dqn_solver.remember(state, action, reward, state_next, terminal)
            state = state_next
            if terminal:
                print "Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate) + ", score: " + str(step)
                score_logger.add_score(step, run)
                break
            dqn_solver.experience_replay()


if __name__ == "__main__":
    cartpole()

In [ ]:
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.compat.v1.enable_eager_execution()


import random
import gym
import numpy as np
from collections import deque
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pylab
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
import os

ENV_NAME = "CartPole-v1"

GAMMA = 0.1
LEARNING_RATE = 0.001

MEMORY_SIZE = 100000
MIN_MEMORY_SIZE = 2000
BATCH_SIZE = 64

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.01
EXPLORATION_DECAY = 0.9995

from environment import GridWorld

now = '30x40_randomAgent_fixedTarget_nomeans'

scores = []
averages = []
episodes = []

debug_mode = False
show_graph_every = 1
means = False
resume = True


env = GridWorld(show_graph_every, debug_mode, means)


size = 1200



class DQNSolver:

    def __init__(self, observation_space, action_space):
        self.exploration_rate = EXPLORATION_MAX
        self.action_space = action_space
        self.memory = deque(maxlen=MEMORY_SIZE)
        if resume:
            self.model = keras.models.load_model('models/'+now+'.h5', custom_objects=None, compile=True)
            print('Model loaded')
            self.exploration_rate = EXPLORATION_MIN
            self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))
            return
        self.model = Sequential()
        self.model.add(Dense(size, input_shape=(observation_space,), activation="relu"))
        self.model.add(Dense(size, activation="relu"))
        self.model.add(Dense(self.action_space, activation="linear"))
        self.model.compile(loss="mse", optimizer=Adam(lr=LEARNING_RATE))
        print('Model created')

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):        
        if np.random.rand() < self.exploration_rate:
            return random.randrange(self.action_space)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def save(self):
        self.model.save('models/'+now+'.h5')   
        print('model saved')
    
    def experience_replay(self, run):
        if len(self.memory) < MIN_MEMORY_SIZE:
            return
        batch = random.sample(self.memory, BATCH_SIZE)
        for state, action, reward, state_next, terminal in batch:
            q_update = reward
            if not terminal:
                q_update = (reward + GAMMA * np.amax(self.model.predict(state_next)[0]))
            q_values = self.model.predict(state)
            q_values[0][action] = q_update
            self.model.fit(state, q_values, verbose=0)                
        self.exploration_rate *= EXPLORATION_DECAY
        self.exploration_rate = max(EXPLORATION_MIN, self.exploration_rate)


def cartpole():
    env = GridWorld(show_graph_every, debug_mode, means)
    #observation_space = env.observation_space.shape[0]
    action_space = 6
    observation_space = size
    dqn_solver = DQNSolver(observation_space, action_space)
    run = 0
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        step = 0
        while True:
            step += 1
            action = dqn_solver.act(state)
            observe_past, reward, total_reward, state_next, terminal = env.step(action)
            #reward = reward if not terminal else -reward
            state_next = np.reshape(state_next, [1, observation_space])
            dqn_solver.remember(state, action, reward, state_next, terminal)
            state = state_next
            if terminal:
                print ("Run: " + str(run) + ", exploration: " + str(dqn_solver.exploration_rate)[:4] + ", score: " + str(total_reward)[:4] + ' step: '+ str(step))
                PlotModel(total_reward, run)                
                break
            dqn_solver.experience_replay(run)
        if run % 20 == 0:
            dqn_solver.save()



def cartpole_test():
    #observation_space = env.observation_space.shape[0]
    global show_graph_every
    show_graph_every = 1
    env = GridWorld(show_graph_every, debug_mode, means)
    action_space = 5
    observation_space = size
    model = keras.models.load_model('models/'+now+'.h5', custom_objects=None, compile=True)
    print('Model loaded')
    run = 0
    while True:
        run += 1
        state = env.reset()
        state = np.reshape(state, [1, observation_space])
        step = 0
        while True:
            step += 1
            action = np.argmax(model.predict(state)[0])
            observe_past, reward, total_reward, state_next, terminal = env.step(action)
            state_next = np.reshape(state_next, [1, observation_space])
            state = state_next
            if terminal:
                print ("Test: " + str(run) + ", score: " + str(total_reward)[:4] + ' step: '+ str(step))
                break
         
            
            
            
            
            
            
def PlotModel(score, episode_number):
    scores.append(score)
    averages.append(sum(scores[-50:]) / len(scores[-50:]))
    episodes.append(episode_number)
    pylab.plot(episodes, scores, 'b')
    pylab.plot(episodes, averages, 'r')
    pylab.ylabel('Score', fontsize=18)
    pylab.xlabel('Games', fontsize=18)
    file_name = "breakout_model_{}".format(now)
    name = file_name + '.png'
    try:
        if not os.path.exists('training_images'): os.makedirs('training_images')
        pylab.savefig('training_images/'+name)

    except OSError as e:
        print(e)
        pass
    return    
            
            
            
            
            
            
            
            
            
            
            
            
            

if __name__ == "__main__":
    cartpole()
    #cartpole_test()

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Model loaded
Run: 1, exploration: 0.01, score: 38.1 step: 9
Run: 2, exploration: 0.01, score: 40.3 step: 10
Run: 3, exploration: 0.01, score: 44.1 step: 25
Run: 4, exploration: 0.01, score: 40.1 step: 9
Run: 5, exploration: 0.01, score: 44.1 step: 25
Run: 6, exploration: 0.01, score: 41.7 step: 17
Run: 7, exploration: 0.01, score: 41.0 step: 14
Run: 8, exploration: 0.01, score: 43.4 step: 23
Run: 9, exploration: 0.01, score: 42.2 step: 19
Run: 10, exploration: 0.01, score: 41.0 step: 14
Run: 11, exploration: 0.01, score: 43.2 step: 30
Run: 12, exploration: 0.01, score: 43.1 step: 22
Run: 13, exploration: 0.01, score: 41.2 step: 15
Run: 14, exploration: 0.01, score: 41.5 step: 16
Run: 15, exploration: 0.01, score: 43.1 step: 22
Run: 16, exploration: 0.01, score: 42.8 step: 21
Run: 17, exploration: 0.01, score: 39.9 step: 6
Run: 18, exploration: 0.01, score: 43.8 step: 24
Run: 19, exploration: 0.01, scor

Run: 164, exploration: 0.01, score: 45.6 step: 29
Run: 165, exploration: 0.01, score: 44.8 step: 27
Run: 166, exploration: 0.01, score: 42.2 step: 19
Run: 167, exploration: 0.01, score: 41.8 step: 24
Run: 168, exploration: 0.01, score: 43.8 step: 24
Run: 169, exploration: 0.01, score: 39.8 step: 5
Run: 170, exploration: 0.01, score: 43.8 step: 24
Run: 171, exploration: 0.01, score: 44.8 step: 27
Run: 172, exploration: 0.01, score: 41.0 step: 14
Run: 173, exploration: 0.01, score: 44.5 step: 26
Run: 174, exploration: 0.01, score: 39.8 step: 5
Run: 175, exploration: 0.01, score: 40.4 step: 11
Run: 176, exploration: 0.01, score: 40.3 step: 10
Run: 177, exploration: 0.01, score: 40.3 step: 10
Run: 178, exploration: 0.01, score: 40.8 step: 13
Run: 179, exploration: 0.01, score: 40.8 step: 13
Run: 180, exploration: 0.01, score: 43.4 step: 23
model saved
Run: 181, exploration: 0.01, score: 43.4 step: 23
Run: 182, exploration: 0.01, score: 40.3 step: 10
Run: 183, exploration: 0.01, score: 36.1

Run: 327, exploration: 0.01, score: 45.6 step: 29
Run: 328, exploration: 0.01, score: 43.8 step: 24
Run: 329, exploration: 0.01, score: 41.0 step: 14
Run: 330, exploration: 0.01, score: 40.6 step: 12
Run: 331, exploration: 0.01, score: 41.5 step: 16
Run: 332, exploration: 0.01, score: 37.2 step: 7
Run: 333, exploration: 0.01, score: 40.5 step: 20
Run: 334, exploration: 0.01, score: 42.0 step: 18
Run: 335, exploration: 0.01, score: 46.3 step: 31
Run: 336, exploration: 0.01, score: 43.4 step: 23
Run: 337, exploration: 0.01, score: 42.0 step: 18
Run: 338, exploration: 0.01, score: 40.6 step: 12
Run: 339, exploration: 0.01, score: 34.8 step: 27
Run: 340, exploration: 0.01, score: 44.5 step: 26
model saved
Run: 341, exploration: 0.01, score: 42.0 step: 18
Run: 342, exploration: 0.01, score: 41.5 step: 16
Run: 343, exploration: 0.01, score: 39.8 step: 4
Run: 344, exploration: 0.01, score: 43.4 step: 23
Run: 345, exploration: 0.01, score: 41.0 step: 14
Run: 346, exploration: 0.01, score: 39.8

...

Run: 1139, exploration: 0.01, score: 45.2 step: 28
Run: 1140, exploration: 0.01, score: 33.1 step: 33
model saved
Run: 1141, exploration: 0.01, score: -9.0 step: 50
Run: 1142, exploration: 0.01, score: 39.8 step: 2
Run: 1143, exploration: 0.01, score: 40.1 step: 9
Run: 1144, exploration: 0.01, score: 42.2 step: 19
Run: 1145, exploration: 0.01, score: 41.5 step: 16
Run: 1146, exploration: 0.01, score: 40.4 step: 11
Run: 1147, exploration: 0.01, score: 41.2 step: 15
Run: 1148, exploration: 0.01, score: 43.8 step: 24
Run: 1149, exploration: 0.01, score: 40.4 step: 11
Run: 1150, exploration: 0.01, score: 41.7 step: 17
Run: 1151, exploration: 0.01, score: 43.1 step: 22
Run: 1152, exploration: 0.01, score: 46.3 step: 31
Run: 1153, exploration: 0.01, score: 42.2 step: 19
Run: 1154, exploration: 0.01, score: 40.6 step: 12
Run: 1155, exploration: 0.01, score: 44.1 step: 25
Run: 1156, exploration: 0.01, score: 40.0 step: 8
Run: 1157, exploration: 0.01, score: 41.5 step: 16
Run: 1158, exploration

Run: 1299, exploration: 0.01, score: 40.0 step: 8
Run: 1300, exploration: 0.01, score: 39.8 step: 5
model saved
Run: 1301, exploration: 0.01, score: 43.8 step: 24
Run: 1302, exploration: 0.01, score: 41.2 step: 15
Run: 1303, exploration: 0.01, score: 44.8 step: 27
Run: 1304, exploration: 0.01, score: 40.1 step: 9
Run: 1305, exploration: 0.01, score: 40.3 step: 10
Run: 1306, exploration: 0.01, score: 42.0 step: 18
Run: 1307, exploration: 0.01, score: 41.5 step: 16
Run: 1308, exploration: 0.01, score: 44.5 step: 26
Run: 1309, exploration: 0.01, score: 42.5 step: 20
Run: 1310, exploration: 0.01, score: 44.9 step: 32
Run: 1311, exploration: 0.01, score: 42.5 step: 20
Run: 1312, exploration: 0.01, score: 42.8 step: 21
Run: 1313, exploration: 0.01, score: 41.5 step: 16
Run: 1314, exploration: 0.01, score: 45.2 step: 28
Run: 1315, exploration: 0.01, score: 43.1 step: 22
Run: 1316, exploration: 0.01, score: 42.8 step: 21
Run: 1317, exploration: 0.01, score: 41.5 step: 16
Run: 1318, exploration

Run: 1459, exploration: 0.01, score: 42.2 step: 19
Run: 1460, exploration: 0.01, score: 41.5 step: 16
model saved
Run: 1461, exploration: 0.01, score: 40.4 step: 11
Run: 1462, exploration: 0.01, score: 43.8 step: 24
Run: 1463, exploration: 0.01, score: 39.5 step: 16
Run: 1464, exploration: 0.01, score: 40.1 step: 9
Run: 1465, exploration: 0.01, score: 41.5 step: 16
Run: 1466, exploration: 0.01, score: 39.8 step: 4
Run: 1467, exploration: 0.01, score: 42.5 step: 20
Run: 1468, exploration: 0.01, score: 43.8 step: 24
Run: 1469, exploration: 0.01, score: 42.2 step: 19
Run: 1470, exploration: 0.01, score: 44.1 step: 25
Run: 1471, exploration: 0.01, score: 41.5 step: 16
Run: 1472, exploration: 0.01, score: 40.0 step: 8
Run: 1473, exploration: 0.01, score: 40.6 step: 12
Run: 1474, exploration: 0.01, score: 40.0 step: 8
Run: 1475, exploration: 0.01, score: 45.6 step: 29
Run: 1476, exploration: 0.01, score: 41.7 step: 17
Run: 1477, exploration: 0.01, score: 40.4 step: 22
Run: 1478, exploration:

Run: 1619, exploration: 0.01, score: 43.4 step: 23
Run: 1620, exploration: 0.01, score: 42.8 step: 21
model saved
Run: 1621, exploration: 0.01, score: 40.4 step: 11
Run: 1622, exploration: 0.01, score: 41.0 step: 14
Run: 1623, exploration: 0.01, score: 39.8 step: 5
Run: 1624, exploration: 0.01, score: 42.0 step: 18
Run: 1625, exploration: 0.01, score: 43.2 step: 28
Run: 1626, exploration: 0.01, score: 39.5 step: 16
Run: 1627, exploration: 0.01, score: 39.8 step: 4
Run: 1628, exploration: 0.01, score: 42.0 step: 18
Run: 1629, exploration: 0.01, score: 42.5 step: 26
Run: 1630, exploration: 0.01, score: 41.2 step: 15
Run: 1631, exploration: 0.01, score: 40.6 step: 12
Run: 1632, exploration: 0.01, score: 38.0 step: 8
Run: 1633, exploration: 0.01, score: 41.2 step: 15
Run: 1634, exploration: 0.01, score: 45.8 step: 32
Run: 1635, exploration: 0.01, score: 40.4 step: 11
Run: 1636, exploration: 0.01, score: 41.4 step: 23
Run: 1637, exploration: 0.01, score: 40.8 step: 13
Run: 1638, exploration

KeyboardInterrupt: 